In [65]:
# Import required libraries for API requests and YAML config parsing
import requests
import json
import yaml
from pathlib import Path
# Import the main extractor class from the installed package
from monday_board_extractor import MondayColumnExtractor

In [66]:
# Automatically look for 'monday_config.yaml' in the current working directory (project root)
config_path = Path.cwd() / "monday_config.yaml"
if not config_path.exists():
    raise FileNotFoundError(f"Config file not found at {config_path}. Please ensure 'monday_config.yaml' is in the project root.")

with config_path.open("r") as f:
    config = yaml.safe_load(f)

api_key = config["monday"]["api_key"]

In [67]:
url = "https://api.monday.com/v2"
headers = {
    "Authorization": api_key,
    "Content-Type": "application/json"
}

In [56]:
board_ids = [4490632304]

extractor = MondayColumnExtractor(api_key, board_ids)
results = extractor.extract()

# To access one dataframe by board name:
for board_data in results:
    print(board_data.name)

data = board_data.data

ACTIVE PROJECTS  *UPDATES DUE 15TH OF EACH MONTH*


In [57]:
df = data[['Item', 'Project Phase', 'DISTRICT', 'PM', 'Project Number', 'Improvement Type', 'Project Address / Location / Limits',
      'Reference Address',  'Latitude\nFrom', 'Longitude\nFrom', 'Latitude\nTo', 'Longitude\nTo',
       'Latitude Midpoint', 'Longitude Midpoint', 'Funding Source']]

In [58]:
def clean_address(raw_address):
    if pd.isna(raw_address) or raw_address.strip() == '':
        return None  # Skip empty
    # Take only first line if there are multiple (from \n)
    address = raw_address.split('\n')[0].strip()

    # If it looks like an intersection, append city/state
    if '&' in address:
        address += ', El Paso, TX'

    # If not already containing "El Paso", assume it's local and add city/state
    if "el paso" not in address.lower():
        address += ', El Paso, TX'

    return address

In [59]:
def clean_address(raw_address):
    if pd.isna(raw_address) or raw_address.strip() == '':
        return None
    address = raw_address.split('\n')[0].strip()
    if '&' in address:
        address = f"Intersection of {address}, El Paso, TX"
    elif "el paso" not in address.lower():
        address += ", El Paso, TX"
    return address

df['Cleaned Address'] = df['Project Address / Location / Limits'].apply(clean_address)

C:\Users\CastroJG\AppData\Local\Temp\ipykernel_1924\3610765679.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned Address'] = df['Project Address / Location / Limits'].apply(clean_address)


In [62]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.geometry import SpatialReference
import pandas as pd

# ArcGIS sign-in (if needed) or anonymous
gis = GIS("home")

# Geocoding function
def geocode_address_arcgis(address):
    try:
        if not address or pd.isna(address):
            return pd.Series([None, None])
        result = geocode(address, out_sr=SpatialReference(4326))
        if result:
            location = result[0]['location']
            return pd.Series([location['y'], location['x']])  # [lat, lon]
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding '{address}': {e}")
        return pd.Series([None, None])

# Apply to ALL rows — no mask
df[['Lat_From_Geocoded', 'Lon_From_Geocoded']] = \
    df['Cleaned Address'].apply(geocode_address_arcgis)

C:\Users\CastroJG\AppData\Local\Temp\ipykernel_1924\1030898045.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Lat_From_Geocoded', 'Lon_From_Geocoded']] = \


In [64]:
df[['Item', 'Project Phase', 'DISTRICT', 'PM', 'Project Number', 'Improvement Type', 'Cleaned Address',
    'Lat_From_Geocoded', 'Lon_From_Geocoded', 'Funding Source']].tail(8)

,Item,Project Phase,DISTRICT,PM,Project Number,Improvement Type,Cleaned Address,Lat_From_Geocoded,Lon_From_Geocoded,Funding Source
17,Fire Station 22 Renovation,Construction,8,Alan Munoz,PCP20FDRENOMSTR,Public Safety,"7901 San Jose, El Paso, TX",31.731669,-106.355817,2019 Public Safety Bond
18,Rancho Del Sol Park Improvements - NIP V,Construction,6,Martin Blancas,PCP23NIPD6A,Parks and Rec,"1100 Ted Houghton, El Paso, TX",31.713291,-106.278192,2012 QOL Bond
19,Komodo Dragon Exhibit,Construction,ZOO,Jesus Palma,PCP13ZOOC01,Parks and Rec,"4001 E. Paisano, El Paso, TX",31.767585,-106.445137,2012 QOL Bond
20,Sun Metro Dyer Mini Brio Stations (EPE),Construction,2,Daniel Carrillo,PCP11MT040,Sun Metro,"Intersection of Dyer & Memphis, El Paso, TX",31.799277,-106.444008,Sun Metro
21,Fire Station 38,Construction,5,Jesus Palma,PCP20FDSTATIO38,Public Safety,Intersection of Pebble Hills Dr. & Tim Foster ...,31.782033,-106.225110,2019 Public Safety Bond
22,Fire Department Special Operations Division,Construction,8,Jesus Lara,PCP20FDSPECTEAM,Public Safety,"222 S Campbell St, El Paso Texas 79901",31.757544,-106.482933,2019 Public Safety Bond
23,Sun Metro Pathways,Construction,CW,Manuel Aguilar,,None,None,NaN,NaN,"Sun Metro Grants, FTA"
24,Hondo Pass CCS,Construction,4,Jesus Palma,PESD00210,Streets,"9135 Stahalla Dr., El Paso, TX",31.871124,-106.435935,ESD
